# Batch size optimization for a multi-product line

## Back story
- A factory produces three products A, B, and C on a single machine.
- Switching between any two products requires setup time and cost.
### Problem
Larger batches reduce setup frequency but increase the holding cost.

Hence, one needs to make an optimal **weekly plan** to minimize the total payment that is materialized by the setup and holding costs.

The manager decides the **weekly plan** that includes a **batch size** (constant for the full week) and the **number of batches** to be made.

## Data

| Product | Weekly Order Size (units) | Setup Cost (\$/setup) | Setup Time (hr) | Holding Cost (\$/units/week) | Production Rate (units/hr) | Minimum Batch Size (units) |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| Bird food | 500 | 100 | .5 | .5 | 100 | 80 |
| Cat food | 300 | 150 | 1  | .4 | 80 | 100 |
| Dog food | 200 | 120 | .8 | .6 | 60 | 90 |

Moreover, we have the maximum machine operating hours = 40 hr/week.

## Modeling

### Indices and parameters
- $i \in \{A,B,C\}$, the index for the product.
- $S_{i}$, the setup cost of the product $i$.
- $W_{i}$, the set up time of the product $i$.
- $H_{i}$, the holding cost ot the product $i$.
- $D_{i}$, the weekly demand of the product $i$.
- $R_{i}$, the production rate of the product $i$.
- $B_{i}$, the minimum batch size for the product $i$.
- $T_{i} = D_{i}/R_{i}$, the time needed per week to meet the demand for product $i$.
- $M$, the maximum machin eoperating hours.

### Decision variables
- $Q_{i}$, the batch size for product $i$.
- $N_{i}$, the number of batches per week for product $i$.
Note that $N_{i} = D_{i}/Q_{i}$ and $T_{i} = D_{i}/R_{i}$.

### Objective function
- To minimize the total cost, which is defined by
$$
Cost = \sum_{i} \left(\quad \underbrace{S_{i}N_{i}}_{\text{setup cost}} + \underbrace{.5 H_{i}Q_{i}}_{\text{averaged holding cost}} \right)
$$

### Constraints
- $\sum_{i} (T_{i} + N_{i}W_{i}) \leq M$, the production time and setup time is less than the maximum machine hours.
- $Q_{i} \geq B_{i}$, the production batch is larger than the minimum batch size.
- $Q_{i}N_{i} = D_{i}$, the total production equals to the order size. **This is nonlinear.**

In [ ]:
from pyscipopt import Model, quicksum
import numpy as np

In [ ]:
prod = ["Bird food", "Cat food", "Dog food"]
p = len(prod)
P = range(p)
S = np.array([100, 150, 120])
W = np.array([.5, 1, .8])
H = np.array([.5, .4, .6])
D = np.array([500, 300, 200])
R = np.array([100, 80, 60])
B = np.array([80, 100, 90])
T = D/R
M = 40

m = Model()
Q = [m.addVar(vtype='C', lb=0, name=f'Q_{prod[i]}') for i in P]
N = [m.addVar(vtype='I', lb=0, name=f'P_{prod[i]}') for i in P]
m.setObjective( quicksum(S[i]*P[i] + .5*H[i]*Q[i] for i in P), sense='minimize')
m.addCons( quicksum(T[i] + W[i]*P[i] for i in P) <= M )
for i in P: m.addCons( Q[i] >= B[i] )
for i in P: m.addCons( Q[i]*N[i] == D[i] )

In [ ]:
m.optimize()

In [ ]:
SOL = m.getBestSol()
Q_opt = [SOL[Q[i]] for i in P]
N_opt = [SOL[N[i]] for i in P]
TMH = np.sum(T + W*N_opt)

for i in P:
    print(f"----- {prod[i]} (Demand = {D[i]} units) ----")
    print(f"{N_opt[i]} batches of size {Q_opt[i]}")
    print(f"Total production = {Q_opt[i]*N_opt[i]} units\n")
print(f"Total machine hours = {TMH} hours")
print(f"Total cost = {m.getObjVal()}$")

---